In [1]:
import torch
from torch import tanh
from torch import nn
import pandas as pd
import numpy as np
from numpy import multiply
from sklearn.utils import shuffle
train_file = '/home/duyongkang/PaperRec/Data/Train_2feature.pkl'
test_file = '/home/duyongkang/PaperRec/Data/Test_2feature.pkl'

In [2]:
graph_input = '/home/duyongkang/PaperRec/Data/Vector/Metapath2vec.pkl'
text_input = '/home/duyongkang/PaperRec/Data/Vector/Glove.pkl'
data_t = pd.read_pickle(text_input)
data_g = pd.read_pickle(graph_input)#header=None,names=['node','vector']
g_dic = data_g[['node','vector']].set_index('node').to_dict()['vector']
t_dic = data_t[['node','vector']].set_index('node').to_dict()['vector']
dic = {}   #key为节点名称，value为特征向量
for key in t_dic:
    try:
        c = np.hstack((g_dic[key], np.array(t_dic[key])))
#         t_dic[key].extend(g_dic[key])
        dic[key] = c
    except KeyError:
        continue

In [3]:
df_train = pd.read_pickle(train_file)
df_test = pd.read_pickle(test_file)
# dic = (pd.read_pickle(feature))[['node','vector']].set_index('node').to_dict()['vector']

In [4]:
mul = []
for i in range(len(df_train)):
    a = df_train.author[i]
    p = df_train.paper[i]
    v = multiply(dic[a],dic[p])
    mul.append(v)
df_train['vector'] = mul

In [5]:
mul = []
for i in range(len(df_test)):
    a = df_test.author[i]
    p = df_test.paper[i]
    v = multiply(dic[a],dic[p])
    mul.append(v)
df_test['vector'] = mul

In [6]:
df_test

,author,paper,label,vector
0,a9997,p1919974,1,"[1.4531602336436e-05, -0.000279393540752192, 0..."
1,a9997,p1918384,0,"[-0.00039554265964520996, -0.0015072114688412,..."
2,a9997,p1292783,0,"[-0.00047680435134015, -0.0003802308623647441,..."
3,a9997,p2006030,0,"[-0.000163973401754435, -0.000877380137771532,..."
4,a9997,p1292738,0,"[-0.0007872267990917, -0.0023097401362852004, ..."
...,...,...,...,...
536945,a1000410,p1875186,0,"[1.17522484205025e-07, -2.714953490646765e-05,..."
536946,a1000410,p1452071,0,"[0.0001168796271958925, -6.931527195226249e-05..."
536947,a1000410,p2004219,0,"[-0.000169374516337785, -7.85235282678755e-05,..."
536948,a1000410,p1452063,0,"[0.00018897052668370002, -0.000108718617912385..."


In [7]:
# 将tensor转换成list
def to_list(y):
    y = y.cpu()
    try:
        y = y.numpy()
    except:
        y = y.detach().numpy()
    y = y.astype(np.float).tolist()
    y = [y[i][0] for i in range(len(y))]
    return y
def HR_score(y_true, y_pred, k, truncate=False):
    n_positive = y_true.tolist().count(1)
    # make sure y_true and y_pred have the same length
    assert len(y_true) == len(y_pred)
    assert k > 0 and n_positive > 0

    # truncate y_true and y_pred according to k and pos num
    if truncate:
        topk = min(k, y_true.tolist().count(1))
    else:
        topk = k
    order = np.argsort(y_pred, kind="stable")[-topk:][::-1]
    y_true = y_true[order]

    return (y_true > 0).sum() / n_positive

def DCG_score(y_true, y_pred, k, truncate=False):
    # make sure y_true and y_pred have the same length
    assert len(y_true) == len(y_pred)
    assert k > 0 and y_true.tolist().count(1) > 0

    # truncate y_true and y_pred according to k and pos num
    if truncate:
        topk = min(k, y_true.tolist().count(1))
    else:
        topk = k
    order = np.argsort(y_pred, kind="stable")[-topk:][::-1]
    y_true = y_true[order]

    # linear gain, not seansitive to position
    # gains = y_true
    # exponential gain, seansitive to position
    gains = 2 ** y_true - 1

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)

    # return DCG@k
    return np.sum(gains / discounts)

def nDCG_score(y_true, y_pred, k):
    best = DCG_score(y_true, y_true, k)
    real = DCG_score(y_true, y_pred, k)
    # return nDCG@k
    return real / best if best != 0 else 0

# %%
def evaluate(y_true, y_pred, k, length=50):
    NDCG, HR, total = 0.0, 0.0, 0

    for i in range(0, len(y_true), length):
        total += 1

        cur_y_true = y_true[i : i+length]
        cur_y_pred = y_pred[i : i+length]

        NDCG += nDCG_score(cur_y_true, cur_y_pred, k)
        HR   += HR_score  (cur_y_true, cur_y_pred, k)
    
    return NDCG / total, HR / total

In [8]:
## 转化成Tensor
# df_train = pd.read_pickle('/home/duyongkang/PaperRec/Data/Glove_train.pkl')
# df_test = pd.read_pickle('/home/duyongkang/PaperRec/Data/Glove_test.pkl')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# df_train = df_train.sort_index(ascending=True)  # 按索引排序
df_train = shuffle(df_train)
x_train = torch.Tensor(df_train.vector)
y_train = torch.from_numpy(df_train.label.values.reshape(-1,1).astype(np.float32))
# 放在GPU中运算

x_train = x_train.to(device)
y_train = y_train.to(device)

x_test = torch.Tensor(df_test.vector)
y_test = torch.from_numpy(df_test.label.values.reshape(-1,1).astype(np.float32))
# 放在GPU中运算
x_test = x_test.to(device)
y_test = y_test.to(device)
## 定义模型

import tensorflow as tf
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin_1 = nn.Linear(164, 1024)
        self.lin_2 = nn.Linear(1024, 1)
        self.activate = nn.Tanh()
    def forward(self, input):  # nn.torch.embedding
        x = self.activate(input)
        x = self.lin_1(x)
        x = self.activate(x)
        x = self.lin_2(x)
        return x
lr = 0.03
def get_model():
    model = Net()
    return model, torch.optim.Adam(model.parameters(), lr=lr)

model, opt = get_model()
loss_fn = nn.BCEWithLogitsLoss()
model = model.to(device)

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
batch = 256*2*2                         #每批训练量
no_of_batches = len(df_train) // batch
epochs = 100                           #轮次
Traindataset = TensorDataset(x_train, y_train)
# Train_dl = DataLoader(Traindataset, batch_size=batch)
for epoch in range(epochs):
    for i in range(no_of_batches):
        x, y = Traindataset[i * batch: i * batch + batch]
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)                 #使用模型预测
        loss = loss_fn(y_pred, y)         #根据结果计算损失
        opt.zero_grad()                   #梯度清零
        loss.backward()                   #反向传播
        opt.step()                        #优化
    print('epoch:', epoch, '   ', 'loss:', loss_fn(model(x_train), y_train))
    
    # model = model.to(device)
    y_pred = model(x_test)
    y_true = np.array(to_list(y_test))
    y_p = np.array(to_list(y_pred))
    ndcg, hr = evaluate(y_true,y_p,5)
    print('NDCG: ',ndcg, 'HR: ', hr)


/home/duyongkang/.conda/envs/gpuenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/duyongkang/.conda/envs/gpuenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/duyongkang/.conda/envs/gpuenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/duyongkang/.conda/envs/gpuenv/lib/pyt

epoch: 0     loss: tensor(0.2958, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


RuntimeError: CUDA out of memory. Tried to allocate 2.05 GiB (GPU 0; 10.92 GiB total capacity; 2.81 GiB already allocated; 678.38 MiB free; 316.81 MiB cached)

In [ ]:
# x_test = torch.Tensor(df_test.vector)
# y_test = torch.from_numpy(df_test.label.values.reshape(-1,1).astype(np.float32))
# # 放在GPU中运算

# x_test = x_test.to(device)
# y_test = y_test.to(device)
# # model = model.to(device)

# y_pred = model(x_test)
# y_true = np.array(to_list(y_test))
# y_p = np.array(to_list(y_pred))

for k in range(5,11):
    ndcg, hr = evaluate(y_true,y_p,k)
    print('k: ', k, '  ','NDCG: ',ndcg, 'HR: ', hr)